---

# Applied Data Science Capstone Project

(edited 12.12.2019, Dirk)  

This notebook is used for sharing results in peer-graded assignments  
of the IBM course *Applied Data Science Capstone* on Coursera.  

This *single* Notebook will be linked in the **3 URL fields** of the Coursera submission.   

**Please refer to the respective notebook sections:**

1. <a href="#item1">**Scraping Toronto Postal Codes from Wikipedia and pre-processing**</a> (<a href="#item1b">see dataframe</a>)  

2. <a href="#item2">**Geolocation of neighborhoods**</a> (<a href="#item2b">see dataframe</a>)  

3. <a href="#item3">**Exploring Toronto neighborhoods**</a> (<a href="#item3b">see clusters</a>)  
  
---

### Library imports and local environment configuration

In [2]:
# Needed for scraping html from website in Skills Network Labs Jupyter environment:
#   Had to restart kernel after installation before file scraping actuall worked!
#   (On my local computer with Anaconda Jupyter notebook, no initial lxml installation or import were necessary.)
!pip install lxml   
#!conda install -c conda-forge lxml --yes

     |████████████████████████████████| 5.8MB 14.3MB/s eta 0:00:01


In [8]:
#!pip install beautifulsoup4  # Used for consistency check of direct html reading

In [2]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install geopy    # Needed to retrieve geolocations

     |████████████████████████████████| 102kB 13.5MB/s ta 0:00:01


In [ ]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab

In [3]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1) # Account for wide collected Neighborhoods columns. (No conflict with the other options?)

import lxml # Needed for reading html from website?
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<a id="item1"></a>
## 1. **Scraping Toronto Postal Codes from Wikipedia and pre-processing**

### 1.1 Get Postal Codes directly as html import with pandas from Wikipedia

In [2]:
# Get Postal Codes of Toronto, Canada
url_wiki = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data_M = pd.read_html(url_wiki, header = 0)

In [3]:
print("Number of list elements in imported html data: ",len(data_M))
#data_M[0][0:3]
df_M = data_M[0]
print("The first element is the needed Postal Code assigned to a dataframe of shape: ", df_M.shape)
df_M.head(4)

Number of list elements in imported html data:  3
The first element is the needed Postal Code assigned to a dataframe of shape:  (287, 3)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village


**Inspect the dataframe:**

In [4]:
print(df_M.columns)
print(df_M.index)
#df_M.describe()
df_M.info()

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')
RangeIndex(start=0, stop=287, step=1)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287 entries, 0 to 286
Data columns (total 3 columns):
Postcode         287 non-null object
Borough          287 non-null object
Neighbourhood    287 non-null object
dtypes: object(3)
memory usage: 6.9+ KB


In [5]:
df_M.dtypes

Postcode         object
Borough          object
Neighbourhood    object
dtype: object

#### Consistency check with *Beautiful Soup*

In [9]:
import requests
from bs4 import BeautifulSoup
res = requests.get(url_wiki)
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0]
df_soup = pd.read_html(str(table), header = 0)[0]

In [10]:
print("Soup looks fine. Shape: ", df_soup.shape)
df_soup.head(5)

Soup looks fine. Shape:  (287, 3)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [11]:
# Also in this scrape result, there is no explicit 'Regent Park' neighborhood in the dataframe, see below.
df_soup.loc[df_soup['Postcode'] == 'M5A']

,Postcode,Borough,Neighbourhood
4,M5A,Downtown Toronto,Harbourfront


In [ ]:
# Another check:  Select-Copy-Paste from website to an ASCI text file showed also to be consistent
#
# The '.txt' file appeared neatly as tab-separated (using ConTEXT editor).
#
#df_check = pd.read_csv('wiki_Toronto_consistency_check.txt', 
#                         sep = '\t', header = None, 
#                         names = ['Postcode', 'Borough' , 'Neighbourhood'])
#df_check.head(5)
#df_check.shape
#df_check.dtypes

### 1.2 Pre-processing with direct html import

**Rename columns:**

In [12]:
df_M.rename(columns = {'Postcode': 'PostalCode', 'Neighbourhood': 'Neighborhood'}, inplace = True)

**Remove not assigned Borough rows:**

In [13]:
df_M['Borough'].value_counts()

Not assigned        77
Etobicoke           44
North York          38
Downtown Toronto    37
Scarborough         37
Central Toronto     17
West Toronto        13
York                9 
East Toronto        7 
East York           6 
Mississauga         1 
Queen's Park        1 
Name: Borough, dtype: int64

In [14]:
df_M['Neighborhood'].value_counts()[0:3]

Not assigned      78
Runnymede         2 
St. James Town    2 
Name: Neighborhood, dtype: int64

There are **77 'Not assigned'** *Borough* entries. Remove those rows and reset index.  
Note also 1 excess *'Not assigned' Neighborhood*, treated below.

In [15]:
df_M.drop(df_M[df_M.Borough == 'Not assigned'].index, inplace = True)
#df_M[df_M.Borough != 'Not assigned']   # Alternatively this retrieves a series without the not assigned rows.
df_M.reset_index(drop = True, inplace = True)
print("Shape of dataframe after remowing rows of not assigned Boroughs: ",df_M.shape)
df_M.head(3)
#df_M['Borough'].value_counts()    # Can check correct row removal.

Shape of dataframe after remowing rows of not assigned Boroughs:  (210, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront


**Manually file the missing Neighborhood for Borough Queen's Park:**

In [16]:
print(df_M.loc[df_M['Neighborhood'] == 'Not assigned'])

  PostalCode       Borough  Neighborhood
5  M7A        Queen's Park  Not assigned


In [17]:
df_M.loc[5,'Neighborhood'] = "Queen's Park"
print("Are both fields the same now? ", df_M.loc[5,'Borough'] == df_M.loc[5,'Neighborhood'])
df_M.loc[[5]]

Are both fields the same now?  True


,PostalCode,Borough,Neighborhood
5,M7A,Queen's Park,Queen's Park


**Group Neighborhoods of same Postal Codes:**

In [18]:
df_M = df_M.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
#df_M.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(list)    # This would collect in a list
print("Shape of grouped dataframe: ", df_M.shape)
df_M.head(5)

Shape of grouped dataframe:  (103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


**Manually file Neighborhood Regent Park:**  

Other than stated in the Coursera task assignment, there is **no explicit record** for Neighborhood *Regent Park* in the table.  

However, when navigating with the mouse pointer to the *M5A Harbourfront Neighborhood* on the Wikipedia page,  
there is a pop-up description for Regent Park. Based on this observation, I *manually* add it here to the Neighborhood.

In [19]:
# A single Postal Code entry M5A, only:
df_M.loc[df_M['PostalCode'] == 'M5A']

,PostalCode,Borough,Neighborhood
53,M5A,Downtown Toronto,Harbourfront


In [20]:
# No entry Regent Park:
df_M.loc[df_M['Neighborhood'] == 'Regent Park']

,PostalCode,Borough,Neighborhood


In [21]:
# A single entry Harbourfront (as a single word):
df_M.loc[df_M['Neighborhood'] == 'Harbourfront']

,PostalCode,Borough,Neighborhood
53,M5A,Downtown Toronto,Harbourfront


In [22]:
df_M.loc[53,'Neighborhood'] = 'Regent Park, Harbourfront'
df_M.loc[[53]]

,PostalCode,Borough,Neighborhood
53,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<a id="item1b"></a>
### 1.3 The Pre-processed Toronto Postal Code dataframe ( **TASK RESULT DISPLAYED HERE** )

In [23]:
df_M.head(5)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [24]:
df_M.tail(5)

,PostalCode,Borough,Neighborhood
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richview Gardens, St. Phillips"
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown"
102,M9W,Etobicoke,Northwest


In [25]:
df_M.shape

(103, 3)

<a id="item2"></a>
## 2. **Geolocation of neighborhoods**

### 2.1 Try Geocoder (**Seems not working**. Doesn't retrieve any coordinates!)

In [27]:
#!pip install geocoder

In [28]:
import geocoder # import geocoder

In [29]:
print(df_M.shape)
df_M.head(3)

(103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"


In [30]:
# Extract a list of our Toronto Postal Codes:
pc = df_M.PostalCode.tolist()
len(pc)
pc[0:10]

['M1B', 'M1C', 'M1E', 'M1G', 'M1H', 'M1J', 'M1K', 'M1L', 'M1M', 'M1N']

In [31]:
# A function to retrieve the coordinates of a Postal Code:
def pc_get_coords(pcode):
    # initialize your variable to None
    lat_lng_coords = None
    limit_trials = 10    # Have only 2500 queries a day on geocoder?
    trials = 0
    # loop until you get the coordinates
    while((lat_lng_coords is None) and (trials < limit_trials)):
        g = geocoder.google('{}, Toronto, Ontario'.format(pcode))
        lat_lng_coords = g.latlng
        trials = trials + 1
    #latitude = lat_lng_coords[0]
    #longitude = lat_lng_coords[1]
    print(trials, lat_lng_coords)
    return lat_lng_coords;

**Seems not to work. Never get any coordinates!**

In [34]:
coords = pc_get_coords('M1G')

10 None


### 2.2 Use the **alternative file sample** from Coursera instead

In [35]:
url_coordinates = 'https://cocl.us/Geospatial_data'
df_coord = pd.read_csv(url_coordinates, header = 0)

In [36]:
print(df_coord.shape)
df_coord.head(4)

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917


In [37]:
df_coord.dtypes

Postal Code    object 
Latitude       float64
Longitude      float64
dtype: object

In [38]:
# Prepare with same lable PostalCode for merging the dataframes:
df_coord.rename(columns = {'Postal Code': 'PostalCode'}, inplace = True)
df_coord.head(2)

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497


<a id="item2b"></a>
### 2.3 Add coordinates to Postal Code dataframe ( **TASK RESULT DISPLAYED HERE** )

In [42]:
df_M = pd.merge(df_M, df_coord)

In [43]:
print("Shape of merged dataframe: ", df_M.shape)
df_M.head(5)

Shape of merged dataframe:  (103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [44]:
df_M.tail(5)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richview Gardens, St. Phillips",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",43.739416,-79.588437
102,M9W,Etobicoke,Northwest,43.706748,-79.594054


In [45]:
df_M.dtypes

PostalCode      object 
Borough         object 
Neighborhood    object 
Latitude        float64
Longitude       float64
dtype: object

In [56]:
# Save a local file version of this prepared dataframe
#df_M.to_csv("Capstone_df_M_index.csv", index=True)

<a id="item3"></a>
## 3. **Exploring Toronto neighborhoods**

### 3.1 Select neighborhoods and inspect on map

Follow the assignment suggestion to restrict the clustering analysis to Boroughs containing the word 'Toronto':  

Draw a copy in *df_Toronto*, inspect the 11 Boroughs therein, and remove those without 'Toronto'.

In [90]:
df_Toronto = df_M.copy()
print(df_Toronto.shape)
print('The dataframe has {} boroughs.'.format(len(df_Toronto['Borough'].unique())))
# Need to split 'Neighborhood' in sub-lists or one-hot-encoded columns?
#print('The dataframe has {} boroughs and {} neighborhoods.'.format(
#    len(df_M['Borough'].unique()),
#    len(df_M['Neighborhood'].unique())
#))
df_Toronto.head(2)

(103, 5)
The dataframe has 11 boroughs.


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497


In [91]:
df_Toronto.groupby('Borough').count()

,PostalCode,Neighborhood,Latitude,Longitude
Borough,,,,
Central Toronto,9,9,9,9
Downtown Toronto,19,19,19,19
East Toronto,5,5,5,5
East York,5,5,5,5
Etobicoke,11,11,11,11
Mississauga,1,1,1,1
North York,24,24,24,24
Queen's Park,1,1,1,1
Scarborough,17,17,17,17


In [112]:
# Generate a filter mask for Boroughs having 'Toronto' in their name:
mask_Toronto = df_Toronto['Borough'].str.contains( pat = 'Toronto').tolist()
mask_Toronto[:] = [not x for x in mask_Toronto] # Logical inversion in order to keep the Torontos!
print(len(mask_Toronto))
mask_Toronto[-10:-1]

103


[False, True, True, True, True, True, True, True, True]

In [113]:
# Drop obsolete Borough rows:
df_Toronto.drop(df_Toronto[mask_Toronto].index, inplace = True)
df_Toronto.reset_index(drop = True, inplace = True)

print("Shape of dataframe after remowing rows of not assigned Boroughs: ",df_Toronto.shape)
print('The dataframe now has {} boroughs left.'.format(len(df_Toronto['Borough'].unique())))
df_Toronto.head(3)

Shape of dataframe after remowing rows of not assigned Boroughs:  (39, 5)
The dataframe now has 4 boroughs.


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572


In [114]:
df_Toronto.groupby('Borough').count()

,PostalCode,Neighborhood,Latitude,Longitude
Borough,,,,
Central Toronto,9,9,9,9
Downtown Toronto,19,19,19,19
East Toronto,5,5,5,5
West Toronto,6,6,6,6


#### Use geopy library to get the latitude and longitude values of Toronto

**From here on following the Course Lab template notebook, *DP0701EN-3-3-2-Neighborhoods-New-York-py-v1.0.ipynb* .**  

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [118]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada, are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Canada, are 43.653963, -79.387207.


#### Create a map of Toronto with *Postal Code-combined* neighborhoods superimposed on top.

In [121]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, pc, borough, neighborhood in \
    zip(df_Toronto['Latitude'], df_Toronto['Longitude'], df_Toronto['PostalCode'], df_Toronto['Borough'], df_Toronto['Neighborhood']):
    label = '{}, {}, {}'.format(pc, borough, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

#### The Folium map seems not to render any display here when sharing the notebook - **please refer to a linked sample screenshot**:

A sample screenshot image of the Toronto neighborhoods map is in the GutHub repository of this Capstone Project:  

[**Toronto Neighborhoods:** *Capstone_map_Toronto_Neighborhoods.png*](https://github.com/dirkvoigt/Coursera_Capstone/blob/master/Capstone_map_Toronto_Neighborhoods.png "Toronto Neighborhoods")

### 3.2 Retrieve venue data from Foursquare

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [122]:
# REMOVED IN PUBLIC VERSION!
#CLIENT_ID = 'your-client-ID' # your Foursquare ID
#CLIENT_SECRET = 'your-client-secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [124]:
# NO PRINT IN PUBLIC VERSION!
print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:


#### Explore the first neighborhood in our dataframe and the functionality of Foursquare queries.

Get that neighborhood's name, latitude and longitudes.

In [127]:
neighborhood_latitude = df_Toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_Toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_Toronto.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


#### Now, let's get the top 100 venues that within a radius of 500 meters of The Beaches.

Establish the GET request URL as **url**.

In [129]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
#url # display URL

Send the GET request and examine the resutls

In [130]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5df15c3e760a7f001bbf9b25'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'labe

All the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [131]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [132]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Glen Stewart Ravine,Other Great Outdoors,43.676300,-79.294784
4,Upper Beaches,Neighborhood,43.680563,-79.292869


And how many venues were returned by Foursquare?

In [133]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


#### Establish a function to repeat the same process to all the selected Postal Codes of Toronto, i.e. the combined Neighborhoods

In [135]:
print("Mind the Foursquare daily query limitations! LIMIT per location here is still LIMIT = ", LIMIT)

Mind the Foursquare daily query limitations! LIMIT per location here is still LIMIT =  100


In [136]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Run the function on each PostalCode, i.e. combined Neighborhoods and create a new dataframe called *Toronto_venues*.

In [137]:
Toronto_venues = getNearbyVenues(names=df_Toronto['Neighborhood'],
                                   latitudes=df_Toronto['Latitude'],
                                   longitudes=df_Toronto['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Regent Park, Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

#### Inspect the resulting dataframe

In [139]:
# Save these Foursquare results to a local file as backup
#Toronto_venues.to_csv("Capstone_Toronto_venues_index__df from Foursquare.csv", index=True)

In [138]:
print(Toronto_venues.shape)
Toronto_venues.head()

(1685, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


Let's check how many venues were returned for each neighborhood

In [140]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"Brockton, Exhibition Place, Parkdale Village",22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",15,15,15,15,15,15
"Cabbagetown, St. James Town",43,43,43,43,43,43
Central Bay Street,84,84,84,84,84,84
"Chinatown, Grange Park, Kensington Market",94,94,94,94,94,94
Christie,17,17,17,17,17,17


#### Let's find out how many unique categories can be curated from all the returned venues

In [141]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 232 uniques categories.


### 3.3 Analyze Each Neighborhood

In [142]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

And let's examine the new dataframe size.

In [143]:
Toronto_onehot.shape

(1685, 232)

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [144]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.00,0.000000,0.010000,0.000000,0.030000,0.000000,0.0000,0.000000,0.000000,0.000000,0.030000,0.000000,0.030000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.01,0.020000,0.000000,0.000000,0.01,0.030000,0.010000,0.00,0.000000,0.050000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.070000,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.020000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.010000,0.000000

#### Let's confirm the new size

In [146]:
Toronto_grouped.shape

(38, 232)

#### Let's print each neighborhood along with the top 5 most common venues

In [148]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0  Coffee Shop      0.07
1  Café             0.05
2  Steakhouse       0.04
3  Thai Restaurant  0.04
4  Bakery           0.03


----Berczy Park----
                venue  freq
0  Coffee Shop         0.09
1  Steakhouse          0.04
2  Café                0.04
3  Beer Bar            0.04
4  Seafood Restaurant  0.04


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0  Coffee Shop     0.09
1  Café            0.09
2  Breakfast Spot  0.09
3  Yoga Studio     0.05
4  Intersection    0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Smoke Shop          0.06
1  Garden Center       0.06
2  Skate Park          0.06
3  Farmers Market      0.06
4  Light Rail Station  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0  Airport Lounge    0.13
1  Airport Service   0.13
2  Ai

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [149]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [200]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,Asian Restaurant,Salad Place,Restaurant,Bar,Bakery,Burger Joint
1,Berczy Park,Coffee Shop,Beer Bar,Steakhouse,Bakery,Farmers Market,Cocktail Bar,Seafood Restaurant,Cheese Shop,Café,Breakfast Spot
2,"Brockton, Exhibition Place, Parkdale Village",Breakfast Spot,Coffee Shop,Café,Yoga Studio,Pet Store,Restaurant,Italian Restaurant,Intersection,Burrito Place,Bar
3,Business Reply Mail Processing Centre 969 Eastern,Skate Park,Burrito Place,Recording Studio,Fast Food Restaurant,Auto Workshop,Farmers Market,Spa,Pizza Place,Restaurant,Smoke Shop
4,"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",Airport Service,Airport Terminal,Airport Lounge,Sculpture Garden,Rental Car Location,Harbor / Marina,Boat or Ferry,Bar,Airport Gate,Airport Food Court


In [201]:
neighborhoods_venues_sorted.shape

(38, 11)

### 3.4 Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [154]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [168]:
print(len(kmeans.labels_))
kmeans.labels_

38


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2,
       0, 4, 0, 0, 0, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [197]:
# Mind this last row's Neighborhood! It received now 10 ten venue ratings!
# This causes dataframe merging problems below, i.e. nan entries in 'Cluster Labels' and float values instead of integers.
print(df_Toronto.shape)
df_Toronto.loc[38]

(39, 5)


PostalCode      M9A             
Borough         Downtown Toronto
Neighborhood    Queen's Park    
Latitude        43.6679         
Longitude      -79.5322         
Name: 38, dtype: object

In [202]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = df_Toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0.0,Other Great Outdoors,Pub,Trail,Health Food Store,Wings Joint,Diner,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0.0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Restaurant,Bookstore,Bubble Tea Shop,Caribbean Restaurant,Brewery
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0.0,Sandwich Place,Park,Sushi Restaurant,Fast Food Restaurant,Steakhouse,Ice Cream Shop,Fish & Chips Shop,Burrito Place,Pizza Place,Food & Drink Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0.0,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Bookstore,Sandwich Place,Brewery,Cheese Shop
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2.0,Park,Gym / Fitness Center,Bus Line,Swim School,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [203]:
print(Toronto_merged.shape)
Toronto_merged.dtypes

(39, 16)


PostalCode                object 
Borough                   object 
Neighborhood              object 
Latitude                  float64
Longitude                 float64
Cluster Labels            float64
1st Most Common Venue     object 
2nd Most Common Venue     object 
3rd Most Common Venue     object 
4th Most Common Venue     object 
5th Most Common Venue     object 
6th Most Common Venue     object 
7th Most Common Venue     object 
8th Most Common Venue     object 
9th Most Common Venue     object 
10th Most Common Venue    object 
dtype: object

In [204]:
# Odd: In the course lab notebook, the kmeans labels remained integer in the dataframe!
# Maybe because the last row didn't get any cluster assignment?
print("Last dataframe row is not assigned a Cluster: ", Toronto_merged['Cluster Labels'][38])
Toronto_merged.loc[38]

Last dataframe row is not assigned a Cluster:  nan


PostalCode                M9A             
Borough                   Downtown Toronto
Neighborhood              Queen's Park    
Latitude                  43.6679         
Longitude                -79.5322         
Cluster Labels            NaN             
1st Most Common Venue     NaN             
2nd Most Common Venue     NaN             
3rd Most Common Venue     NaN             
4th Most Common Venue     NaN             
5th Most Common Venue     NaN             
6th Most Common Venue     NaN             
7th Most Common Venue     NaN             
8th Most Common Venue     NaN             
9th Most Common Venue     NaN             
10th Most Common Venue    NaN             
Name: 38, dtype: object

In [215]:
# Remind from above: Dataframe Toronto_venues holds or findings queried from Foursquare:
print(Toronto_venues.shape)
Toronto_venues.head(4)

(1685, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors


In [216]:
# Check for the missing venu findings for Queen's Park:
Toronto_venues.loc[Toronto_venues['Neighborhood'] == "Queen's Park"]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


**It seems that Postal Code M9A, Queen's Park, may be a bit boring? - no venues and therefore no cluster assignment.**  
Indeed, this neighborhood didn't appear in the Toronto_venues dataframe!  


Drop that row from the dataframe for further visualizaton.

In [205]:
print(Toronto_merged.shape)
Toronto_merged = Toronto_merged.drop([38])
print(Toronto_merged.shape)

(39, 16)
(38, 16)


In [206]:
# Change it manually here:
#Toronto_merged.astype({'Cluster Labels': 'int32'}).dtypes
#Toronto_merged = Toronto_merged['Cluster Labels'].astype(np.int32, inplace=True)
# Toronto_merged.astype({'Cluster Labels': 'int32'}) # This is not inplace. Cannot set inpcale = True here.
Toronto_merged = Toronto_merged.astype({'Cluster Labels': 'int32'})

In [207]:
print(Toronto_merged.shape)
Toronto_merged.dtypes

(38, 16)


PostalCode                object 
Borough                   object 
Neighborhood              object 
Latitude                  float64
Longitude                 float64
Cluster Labels            int32  
1st Most Common Venue     object 
2nd Most Common Venue     object 
3rd Most Common Venue     object 
4th Most Common Venue     object 
5th Most Common Venue     object 
6th Most Common Venue     object 
7th Most Common Venue     object 
8th Most Common Venue     object 
9th Most Common Venue     object 
10th Most Common Venue    object 
dtype: object

In [208]:
# Save these cluster results to a local file as backup
#Toronto_merged.to_csv("Capstone_Toronto_merged_cluster dataframe_index_CLEANED.csv", index=True)

Finally, let's visualize the resulting clusters

In [210]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### The Folium map seems not to render any display here when sharing the notebook - **please refer to a linked sample screenshot**:

A sample screenshot image of the clustered Toronto neighborhoods map is in the GutHub repository of this Capstone Project:  

[**Clustered Toronto Neighborhoods:** *Capstone_map_Toronto_Neighborhoods_clustered.png*](https://github.com/dirkvoigt/Coursera_Capstone/blob/master/Capstone_map_Toronto_Neighborhoods_clustered.png "Clustered Toronto Neighborhoods")

In [ ]:
#[I'm an inline-style link](https://www.google.com)
# This doesn't render as the GitHub link is for the repository viewing page?
#![Clustered Toronto Neighborhoods](https://github.com/dirkvoigt/Coursera_Capstone/blob/master/Capstone_map_Toronto_Neighborhoods_clustered.png "Clustered Toronto Neighborhoods")
#<img style = "float: left;" src = "https://github.com/dirkvoigt/Coursera_Capstone/blob/master/Capstone_map_Toronto_Neighborhoods_clustered.png">

<a id="item3b"></a>
### 3.5 Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 0  - ***The Food Cluster*** - Where you won't need to search long for food.

In [222]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1,2] + list(range(6, Toronto_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,The Beaches,Other Great Outdoors,Pub,Trail,Health Food Store,Wings Joint,Diner,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant
1,East Toronto,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Restaurant,Bookstore,Bubble Tea Shop,Caribbean Restaurant,Brewery
2,East Toronto,"The Beaches West, India Bazaar",Sandwich Place,Park,Sushi Restaurant,Fast Food Restaurant,Steakhouse,Ice Cream Shop,Fish & Chips Shop,Burrito Place,Pizza Place,Food & Drink Shop
3,East Toronto,Studio District,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Bookstore,Sandwich Place,Brewery,Cheese Shop
5,Central Toronto,Davisville North,Hotel,Pizza Place,Breakfast Spot,Sandwich Place,Food & Drink Shop,Park,Gym,Clothing Store,Comfort Food Restaurant,Deli / Bodega
6,Central Toronto,North Toronto West,Clothing Store,Coffee Shop,Sporting Goods Shop,Park,Chinese Restaurant,Cosmetics Shop,Mexican Restaurant,Burger Joint,Rental Car Location,Restaurant
7,Central Toronto,Davisville,Pizza Place,Dessert Shop,Sandwich Place,Italian Restaurant,Gym,Café,Sushi Restaurant,Coffee Shop,Fried Chicken Joint,Farmers Market
9,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West",Coffee Shop,Pub,Fried Chicken Joint,Light Rail Station,Restaurant,Liquor Store,Sports Bar,Bagel Shop,Supermarket,Pizza Place
11,Downtown Toronto,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Pizza Place,Flower Shop,Italian Restaurant,Pub,Bakery,Café,Liquor Store,Deli / Bodega
12,Downtown Toronto,Church and Wellesley,Sushi Restaurant,Coffee Shop,Gay Bar,Japanese Restaurant,Restaurant,Mediterranean Restaurant,Gastropub,Gym,Hotel,Café


#### Cluster 1 - ***The Park Cluster*** - Need a walk after the meal?

In [223]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1,2] + list(range(6, Toronto_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Downtown Toronto,Rosedale,Park,Trail,Playground,Cuban Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Diner
23,Central Toronto,"Forest Hill North, Forest Hill West",Park,Jewelry Store,Trail,Sushi Restaurant,Cupcake Shop,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store


#### Cluster 2 - ***The Sports Cluster*** - Before the meal.

In [224]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1,2] + list(range(6, Toronto_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,Lawrence Park,Park,Gym / Fitness Center,Bus Line,Swim School,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


#### Cluster 3 - No decisive opinion on this cluster.

In [225]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1,2] + list(range(6, Toronto_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,Roselawn,Health & Beauty Service,Garden,Wings Joint,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


#### Cluster 4 - No decisive opinion on this cluster

In [226]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1,2] + list(range(6, Toronto_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,"Moore Park, Summerhill East",Restaurant,Playground,Intersection,Wings Joint,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store


Clusters 3 and 4 are single Postal Code areas each.  
Clustering made them distinct from the other clusters,  
thus it may be worth (another time) to get more information from tips, e.g. about very special or well rated restaurants, shops, facilities.